## Funktionen

> Potenzielles TODO: Zu jeder hier definierten Funktion schreiben in welchem Notebook sie verwendet werden.

Neben den Importen werden auch drei Funktionen definiert, die über die verwendeten Notebooks hinweg benötigt werden. Für die Verwendung muss ebenfalls dieses Notebook zuerst ausgeführt werden.

> %run Util/01_functions.ipynb

> TODO: Darstellungsweise der Parameter und Rückgabewerte evtl. anpassen
> TODO: Begriff "situation" hat ihm im Verlauf der Arbeit nicht verstanden

Als Nächstes wird eine Funktion definiert, die die Suche innerhalb einer ``sequence`` ermöglicht. Diese ``sequence`` ist eine Liste bestehend aus Mengen. In diesen Mengen stehen die Spielsituationen in einer der drei zuvor erwähnten Darstellungsweisen. Bei der ``situation`` muss darauf geachtet werden, dass sie die gleiche Darstellung besitzt wie die Elemente in der ``sequence``. Sofern die Funktion ``find_situation_in_sequence`` die Spielsituation in einer der Mengen findet, wird der Mengenindex der Liste zurückgegeben. Andernfalls lautet der Rückgabewert -1.

In [ ]:
def find_situation_in_sequence(situation, sequence):
    for i in range(len(sequence)):
        if situation in sequence[i]:
            return i
    return -1

## Import der Daten

Für die Bestimmung der Züge der KI werden die $S_n$ Mengen verwendet, die im Notebook ``02_calculation.ipynb`` berechnet werden.
Eine Erklärung worum es sich hierbei handelt, findet sich in diesem Notebook.
Die Ergebnisse der Berechnung werden mittels `pickle` serialisiert und in einer ZIP-Datei komprimiert abgespeichert. Zur Verwendung wird das Archiv entpackt und 
die Daten eingelesen. 
Weitere Informationen zum Inhalt der Datei befinden sich ebenfalls im Notebook ``calculation.ipynb``.   

Die Funktion `load_data(filename)` erhält den Dateinamen (``filename``) als Parameter und gibt die Endspieldaten zurück.
Die Rückgabe besteht aus einer Liste von verwendeten Figuren `piece_list` und einer Liste von Mengen `s_n_sequence_integers`, welche die Situationen als Integer enthalten. 
Diese Mengen enthalten jedoch nur die einzigartigen (ungespiegelten) Spielsituationen, für eine Auswertung, müssen sie noch gespiegelt werden.

In [ ]:
def load_data(filename):
    s_n_sequence_integers = []

    with ZipFile("S_n_Results/" + filename + ".chessAI", "r") as zipped:
        with zipped.open("piece_list.pickle") as piece_list_file:
            piece_list = pickle.loads(piece_list_file.read())
            piece_list_file.close()

        file_list = zipped.namelist()
        file_list.remove("piece_list.pickle")
        for file in file_list:
            with zipped.open(file) as s_n:
                s_n_sequence_integers.append(pickle.loads(s_n.read()))
                s_n.close()
        zipped.close()
    return piece_list, s_n_sequence_integers

## Effiziente Speichernutzung
Um den Arbeitsspeicher effizient zu nutzen, können Spielsituationen nicht als Objekte der ``chess`` Bibliothek gespeichert werden.
Einer ausgabe von `sys.getsizeof` zufolge benötigt ein Board-Objekt 48 Byte Speicher. Das ist bereits doppelt so viel, wie die 24 Byte eines Standard-Integers.
`sys.getsizeof` bestimmt jedoch nur die Größe des eigentlichen Objektes. Verwaltet dieses Pointer auf andere Objekte, ist die tatsächliche Größe höher.
Um nicht pro Stellung ein Objekt mit >=48 Byte zu benötigen werden die Positionen der Figuren als Binärer-Wert kodiert und als Integer gespeichert. 

Die Funktion `to_integer` berechnet für ein `board` und eine Liste der darauf befindlichen Figuren `piece_list` diese Zahl.
Es ist wichtig die Liste der Figuren als Übergabeparameter zu erhalten und nicht zu berechnen, da die Position einer Figur in der Liste mit der Position im Binärcode übereinstimmt.
Bei der Dekodierung wird aus dieser Position die Figur bestimmt. 
Das geringwertigste Bit speichert die Information, welcher Spieler am Zug ist.
Die folgenden (pro Figur 6) Bits kodieren die Zahl, welche der Position einer Figur auf dem Spielfeld entspricht.
Die Reihenfolge der Figuren entspricht deren Reihenfolge in der `piece_list`.
  
Die Funktion gibt einen Integer mit 24 oder 32 Byte Größe (32 Byte bei Bauern in der Stellung) zurück, welcher die Spielsituation repräsentiert.

### Beispiel der Kodierung:
Für das Board:
```
. . . . k . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . Q . .
. . . . K . . .
```
Mit der zugehörigen piece_list:
`[chess.Piece.from_symbol('k'), chess.Piece.from_symbol('K'), chess.Piece.from_symbol('Q')]`

Wird zuerst die `piece_map` bestimmt:
`{4: Piece.from_symbol('K'), 60: Piece.from_symbol('k'), 13: Piece.from_symbol('Q')}`
(Hinweis: Für dieses Beispiel wurde die Reihenfolge der Elemente in der ``piece_map`` vertauscht)

Anschließend wird der Spieler am Zug in das erste Bit kodiert.
Dieses ist somit `0` oder `1`.

Zum Kodieren der Positionen wird über die Inhalte der Piece Map iteriert. 
Das erste Element ist der weiße König `K`.
Für dieses wird der Index in der `piece_list` bestimmt. Dieser ist `0`.
Die Position 4 (aus der `piece_map`) wird nun an die Positionen 1 bis 7 (Index 0 in `piece_list`) der Binärzahl zu `0000100` kodiert.
Für das nächste Element (`k`) wird die Position (`60`) an die Stellen 8 bis 14 (Index 1  in `piece_list`) zu `0111100` kodiert.
Für das letzt Element (`Q`) wird die Position (`13`) an die Stellen 15 bis 21 (Index 2  in `piece_list`) zu `0001101` kodiert.

Daraus entsteht die folgende Binärzahl: `001101011110000001001`, welche von der Funktion als Integer `441353` zurückgegeben wird.


Die Funktion `to_board` wandelt einen Integer und eine Liste von Figuren `piece_list` in einer Spielsituation in ein Board-Objekt.
Das geringwertigste Bit, wird in die Information welcher Spieler am Zug ist konvertiert.
Die verbleibenden Bits werden nacheinander zu einer Zahl gewandelt, welche einem Spielfeld (`square`) entspricht. 

Die bestimmten Informationen über Zug und Piece-Map werden einem neuen Board-Objekt hinzugefügt, welches von der Funktion zurückgegeben wird.
